In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install datasets
!pip install transformers
!pip install peft
!pip install evaluate
!pip install -U datasets==2.20.0 pyarrow==15.0.2 transformers==4.44.2 evaluate==0.4.2 --no-cache-dir
!pip install -q datasets evaluate accelerate scikit-learn pandas matplotlib
!pip install -U "transformers>=4.41" accelerate safetensors
!pip install -U bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 257.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 305.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 236.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 194.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 199.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 363.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 450.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 175.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 293.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: fss

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 132.8 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 113.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.11.0
    Uninstalling accelerate-1.11.0:
      Successfully uninstalled accelerate-1.11.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 13.4 MB/s eta 0:00:00:00:0100:01


In [3]:
!pip install --upgrade --no-cache-dir git+https://github.com/Shannu3766/bi_influence.git

  Cloning https://github.com/Shannu3766/bi_influence.git to /tmp/pip-req-build-i2y3opzu
  Running command git clone --filter=blob:none --quiet https://github.com/Shannu3766/bi_influence.git /tmp/pip-req-build-i2y3opzu
  Resolved https://github.com/Shannu3766/bi_influence.git to commit 0bfad85a0a373b96ec3af51160622924d109e3be
  Preparing metadata (setup.py) ... done
  Created wheel for adaptive_lora: filename=adaptive_lora-2.1.0-py3-none-any.whl size=7940 sha256=c1cd5dd2ae164006070b81b59b627341eb660c6bf223f18b990183a8ef46b2e2
  Stored in directory: /tmp/pip-ephem-wheel-cache-ncurbw38/wheels/f8/5f/fc/97210c723ddb66cb0b5c21449c734301ecc806d8d629ad0829
Successfully built adaptive_lora


In [4]:
import warnings
import os
import logging
os.environ["WANDB_DISABLED"] = "true"
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "true"
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings("ignore")
# logging.set_verbosity_error()

In [6]:
from datasets import load_dataset, DatasetDict, Dataset

from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType
from torch.utils.data import DataLoader
# from adaptive_lora.callbacks import AdaptiveLoRACallback
import numpy as np
print("torch:", torch.__version__)
print("cuda devices:", torch.cuda.device_count())

ModuleNotFoundError: Could not import module 'AutoTokenizer'. Are this object's requirements defined correctly?

In [ ]:
from datasets import load_dataset

In [ ]:
MODEL_ID = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
TASK = "rte"
NUM_LABELS = 2
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
dataset = load_dataset("glue",'rte')
metric = evaluate.load("accuracy")

In [ ]:
from transformers import AutoTokenizer

MODEL_ID = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_ID,
    use_fast=True,
    trust_remote_code=True,
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def preprocess_function(examples):
    return tokenizer(
        examples["sentence1"],
        examples["sentence2"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

encoded_dataset = dataset.map(preprocess_function, batched=True)

In [ ]:
# 1. Load Model
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_ID,
    num_labels=NUM_LABELS,
    load_in_4bit=True,
    device_map="auto",
    trust_remote_code=True
)

model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False # Disable cache for training

# 2. Setup LoRA
target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
lora_config = LoraConfig(
    r=8,                # Increased rank slightly for better capacity
    lora_alpha=32,       # Standard rule: alpha = 2 * rank
    lora_dropout=0.05,
    target_modules=target_modules,
    bias="none",
    task_type="SEQ_CLS"
)
model = get_peft_model(model, lora_config)

In [ ]:
training_args = TrainingArguments(
    output_dir="./TinyLlama_RTE_Fixed",
    eval_strategy="epoch",          # Evaluate every epoch
    save_strategy="epoch",
    learning_rate=2e-4,             # LoRA needs higher LR (2e-5 is for full finetuning)
    per_device_train_batch_size=8,  # Increased batch size
    gradient_accumulation_steps=2,  # Accumulate gradients to simulate batch_size=16
    num_train_epochs=10,            # RTE is small; needs more epochs to converge
    fp16=False,
    bf16=True,
    logging_steps=10,
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    accuracy = (predictions == torch.tensor(labels)).float().mean().item()
    return {"accuracy": accuracy} 

In [ ]:
# # ===============================================================
# # ✅ DataLoader and Adaptive LoRA Callback Setup (Fixed Version)
# # ===============================================================

# from torch.utils.data import DataLoader
# from transformers import DataCollatorWithPadding
# from adaptive_lora.callbacks import AdaptiveLoRACallback

# # -----------------------------
# # 🧹 1. Clean up the dataset
# # -----------------------------
# # Rename label column if necessary
# if "label" in encoded_dataset["train"].column_names:
#     encoded_dataset = encoded_dataset.rename_column("label", "labels")

# # Remove non-tensor columns (raw strings)
# keep_cols = {"input_ids", "attention_mask", "labels", "token_type_ids"}
# for split in encoded_dataset.keys():
#     drop_cols = [c for c in encoded_dataset[split].column_names if c not in keep_cols]
#     if drop_cols:
#         encoded_dataset[split] = encoded_dataset[split].remove_columns(drop_cols)

# # Ensure dataset returns PyTorch tensors
# encoded_dataset.set_format(type="torch")


# data_collator = DataCollatorWithPadding(tokenizer=tokenizer, pad_to_multiple_of=8)

# train_dataset = encoded_dataset["train"]
# eval_dataset = encoded_dataset["validation"]

# # (Optional) use smaller subset for BI scoring to speed up adaptation

# val_dataloader = DataLoader(
#     eval_dataset,
#     batch_size=8,
#     shuffle=False,
#     collate_fn=data_collator
# )

# # -----------------------------
# # 🧩 3. Adaptive LoRA Callback
# # -----------------------------
# adaptive_callback = AdaptiveLoRACallback(
#     val_dataloader=val_dataloader,
#     total_rank=512,   # total LoRA rank budget
#     tau=0.7,          # temperature for softmax allocation
#     verbose=True      # print BI scores and rank allocations
# )

# # -----------------------------
# # 🧾 Debug info
# # -----------------------------
# print("✅ Validation columns:", eval_dataset.column_names)
# print("✅ Example batch keys:", next(iter(val_dataloader)).keys())
# print("✅ Adaptive LoRA callback ready.")


In [ ]:
len(encoded_dataset["train"]),len(encoded_dataset["validation"])

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"].select(range(10)),
    eval_dataset=encoded_dataset["validation"].select(range(10)),
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[adaptive_callback],
)

In [ ]:
trainer.train()

In [ ]:
results = trainer.evaluate()
print(f"\n✅ Final Adaptive LoRA Accuracy on RTE: {results['eval_accuracy']:.4f}")
